# IMPORT

In [1]:
import numpy as np
import pandas as pd

In [2]:
# custom username by pwd
import os
pwd = os.getcwd()
username = pwd[9:pwd.find('\\',9)]

In [3]:
# pd.options.display.max_columns = None
# pd.options.display.max_rows = None

In [4]:
from itertools import groupby
from collections import Counter

In [5]:
# input_path = 'C:\\Users\\'+ username +'\\OneDrive - EXIM BANK OF THAILAND\\Shared Drive บส ตส\\Data\\ปี 2567 Q1\\'
# #result_path = 'C:\\Users\\'+ username +'\\OneDrive - EXIM BANK OF THAILAND\\Shared Drive บส ตส\\2. จัดกลุ่มลูกค้า\\RESULT\\'
# output_path = 'C:\\Users\\'+ username +'\\OneDrive - EXIM BANK OF THAILAND\\Shared Drive บส ตส\\2. จัดกลุ่มลูกค้า\\RESULT\\'
# # input_path = r'C:\Users\tharaphonn\OneDrive - EXIM BANK OF THAILAND\Shared Drive บส ตส\Data\ปี 2566 Q4/'
# # result_path = r'C:\Users\tharaphonn\OneDrive - EXIM BANK OF THAILAND\Shared Drive บส ตส\3. รวมศูนย์\RESULT/'

In [6]:
# check path 
input_path = 'ปี 2567 Q2//'
output_path = 'output//'
# os.mkdir(output_path)

LOGIC

1. ดึงรายชื่อกรรมการมีอำนาจลงนาม และผู้ถือหุ้นจากระบบ ENLITE

2. หาลูกค้าที่มีการอนุมัติ หรือ ทบทวนในช่วงที่เราสนใจจากวงเงินยอดคงค้าง

3. เทียบรายชื่อ กรรมการ/ผู้ถือหุ้น ของลูกค้าที่อยู่ในขอบกับลูกค้าที่เหลือทั้งหมด

# INPUT

In [7]:
#@title ข้อมูลวงเงินยอดคงค้าง
#ไฟล์วงเงินยอดคงค้างจากระบบ share point ปรับไฟล์เป็น .xlsx
outstanding = '06.วงเงินและยอดคงค้าง_มิ.ย.2567.xlsx' #@param {type:'string'}
df_out = pd.read_excel( input_path + outstanding , dtype = str)

c:\Anaconda 3\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [8]:
#@title ข้อมูล ENLITE
#จากระบบ ENLITE
enlite = 'ENLITE_6466_Q2_67_20240712.xlsx' #@param {type:'string'}
df_enl = pd.read_excel( input_path + enlite, dtype = str)

In [9]:
#@title ไฟล์กลุ่มบนระบบ LOS
# ได้รับทุกเดือนจากระบบ LOSCRM ส่ง EMAIL
group = 'จัดกลุ่ม_20240701.xlsx' #@param {type:'string'}
df_g = pd.read_excel( input_path + group , dtype = str)

In [10]:
#@title ไฟล์รวมศูนย์บนระบบ LOSCRM
# ขอจากไฟล์ ระบบ CRM
centralize = 'รวมศูนย์_20240702.xlsx' #@param {type:'string'}
df_cen_p = pd.read_excel( input_path + centralize , dtype = str)

In [11]:
#@title ไฟล์เลขนิติบุคคล รหัสลูกค้า
# ทำไฟล์ ENLITE จากระบบ ENLITE และระบบ AS400
id_cuscode = 'รหัส_เลขนิติ_Q2_67.xlsx' #@param {type:'string'}
df_pm = pd.read_excel( input_path + id_cuscode , dtype= str)

In [12]:
#@title ใส่ขอบเขตการตรวจ
start_date = '2024-01-01' #@param {type:"date"}
end_date = '2024-06-30' #@param {type:"date"}
start_date = pd.to_datetime(start_date, format = '%Y-%m-%d')
end_date = pd.to_datetime(end_date, format = '%Y-%m-%d')

In [13]:
#@title กรอกชื่อไฟล์ output
# ใส่ชื่อไฟล์ขาออก
file_name = 'จัดกลุ่ม_Q2_67.xlsx' #@param {type:'string'}

# CODE

In [14]:
df_cen = pd.merge(df_cen_p ,
                             df_out[['รหัสลูกค้า']].drop_duplicates() ,
                             left_on = 'EXIM_CUSTID' ,
                             right_on = 'รหัสลูกค้า')
df_cen = df_cen.drop(columns = ['รหัสลูกค้า'])

## Choose Customer and Clean Outstanding File

In [15]:
df_out1 = df_out[['รหัสลูกค้า' , 'ผู้ดูแลลูกค้า_(แยกส่วน)' , 'ผู้ดูแลลูกค้า_(ตามฝ่าย)' , 'วันที่อนุมัติ',
                           'วันที่ทบทวนล่าสุด']].drop_duplicates()

In [16]:
# def clean_date_out(a):
#   if pd.isna(a) == False:
#     if a[6:8] == '26':
#       a = a.replace('265' , '256')
#       a = a[:6] + str(int(a[6:]) - 543)
#       a = pd.to_datetime( a , format = '%d/%m/%Y')
#     elif a[6:8] == '25':
#       a = a[:6] + str(int(a[6:]) - 543)
#       a = pd.to_datetime( a , format = '%d/%m/%Y')
#     elif a[6:8] == '20' or a[6:8] == '19':
#       a = pd.to_datetime( a , format = '%d/%m/%Y')
#     elif a[6:7] == '3':
#       a = a[:6] + str(int(a[6:]) -2*543)
#       a = pd.to_datetime( a , format = '%d/%m/%Y')
#     elif a[6:8] == '00':
#       a = np.nan
#   return a

In [17]:
def clean_date_out(a):
  if pd.isna(a) == False:
    if a[6:8] == '26':
      a = a.replace('265' , '256')
      a = a[:6] + str(int(a[6:]) - 543)
      a = pd.to_datetime( a , format = '%d/%m/%Y')
    elif a[6:8] == '25':
      a = a[:6] + str(int(a[6:]) - 543)
      a = pd.to_datetime( a , format = '%d/%m/%Y')
    elif a[6:8] == '20' or a[6:8] == '19':
      a = pd.to_datetime( a , format = '%d/%m/%Y')
    elif a[6:7] == '3':
      a = a[:6] + str(int(a[6:]) -2*543)
      a = pd.to_datetime( a , format = '%d/%m/%Y')
    # elif a[6:8] == '00':
    else: 
      a = np.nan
  return a

In [18]:
date_col = ['วันที่อนุมัติ' , 'วันที่ทบทวนล่าสุด']
for i in range(len(date_col)):
  df_out1[date_col[i]] = df_out1[date_col[i]].apply(lambda x : clean_date_out(x))
  # df_out1[date_col[i]] = pd.to_datetime( df_out1[date_col[i]] , format = '%d/%m/%Y')

In [19]:
df_out1 = df_out1.dropna(subset = ['ผู้ดูแลลูกค้า_(แยกส่วน)'])

In [20]:
def check_date_interval(a , b , start_date , end_date):
  c = 0
  if pd.isna(a) == False:
    if (a >= start_date) and (a <= end_date):
      c+=1
  if pd.isna(b) == False:
    if (b >= start_date) and (b <= end_date):
      c +=1
  return c

In [21]:
# df_out1['วันที่อนุมัติ'] = df_out1['วันที่อนุมัติ'].astype(str)
# df_out1['วันที่อนุมัติ'] = df_out1['วันที่อนุมัติ'].str[0:10]
# df_out1['วันที่อนุมัติ'] = pd.to_datetime( 'วันที่อนุมัติ' , format = '%d/%m/%Y')

In [22]:
df_out1['is_in'] = df_out1.apply(lambda x : check_date_interval(x['วันที่อนุมัติ'] , x['วันที่ทบทวนล่าสุด'] , start_date , end_date) , axis = 1)

In [23]:
# จาก df_out1 ถ้า is_in = 0 เป็น head()

In [24]:
df_hp1 = df_out1[df_out1['is_in'] != 0][['รหัสลูกค้า']].drop_duplicates()

In [25]:
df_hp2 = pd.merge(df_hp1 , df_pm , on = 'รหัสลูกค้า' , how = 'left')

In [26]:
df_hp2['CUCTAXCS'] = df_hp2['CUCTAXCS'].fillna(df_hp2['รหัสลูกค้า'])

In [27]:
df_hp = df_hp2[['CUCTAXCS']].drop_duplicates()

In [28]:
print(len(df_out1['รหัสลูกค้า'].unique()),len(df_hp))

5010 2349


In [29]:
cus_list = df_out1['รหัสลูกค้า'].unique().tolist()
df_out2 = pd.DataFrame()
for i in range(len(cus_list)):
  df_t = df_out1[df_out1['รหัสลูกค้า'] == cus_list[i]]

  date1_min = min(list(df_t['วันที่อนุมัติ']))
  date1_max = max(list(df_t['วันที่อนุมัติ']))

  #date2_min = min(list(df_t['วันที่ทบทวนล่าสุด']))
  date2_max = min(list(df_t['วันที่ทบทวนล่าสุด']))

  df_t1 = df_t.head(1)
  df_t1 = df_t1.drop(columns = ['วันที่อนุมัติ' , 'วันที่ทบทวนล่าสุด'])
  df_t1['วันที่อนุมัติครั้งแรก'] = date1_min
  df_t1['วันที่อนุมัติล่าสุด'] = date1_max
  #df_t1['วันที่ทบทวนครั้งแรก'] = date2_min
  df_t1['วันที่ทบทวนล่าสุด'] = date2_max

  df_out2 = pd.concat([df_out2 , df_t1])

In [30]:
print(len(df_out1) , len(df_out2) , len(df_hp))

8113 5010 2349


## Clean ENLITE File

In [31]:
#df_enl1 = df_enl[[ 'เลขทะเบียนนิติบุคคล (13 หลัก)',
#       'ชื่อนิติบุคคลจดทะเบียนภาษาไทย', 'ชื่อนิติบุคคลจดทะเบียนภาษาอังกฤษ' , 'อำนาจกรรมการ' ,'% ที่ถือหุ้นของ Held By 1', '% ที่ถือหุ้นของ Held By 2',
#       '% ที่ถือหุ้นของ Held By 3', '% ที่ถือหุ้นของ Held By 4',
#       '% ที่ถือหุ้นของ Held By 5', '% ที่ถือหุ้นของ Held By 6',
#       '% ที่ถือหุ้นของ Held By 7', '% ที่ถือหุ้นของ Held By 8',
#       '% ที่ถือหุ้นของ Held By 9', '% ที่ถือหุ้นของ Held By 10',
#       'ชื่อกรรมการ 1', 'ชื่อกรรมการ 2', 'ชื่อกรรมการ 3', 'ชื่อกรรมการ 4',
#       'ชื่อกรรมการ 5', 'ชื่อกรรมการ 6', 'ชื่อกรรมการ 7', 'ชื่อกรรมการ 8',
#       'ชื่อกรรมการ 9', 'ชื่อกรรมการ 10']]

In [32]:
# columns_name_V1
df_enl1 = df_enl[[ 'เลขทะเบียนนิติบุคคล (13 หลัก)',
       'ชื่อนิติบุคคลจดทะเบียนภาษาไทย', 'ชื่อนิติบุคคลจดทะเบียนภาษาอังกฤษ' , 'อำนาจกรรมการ' ,'% ที่ถือหุ้นของ Held By 1', '% ที่ถือหุ้นของ Held By 2',
       '% ที่ถือหุ้นของ Held By 3', '% ที่ถือหุ้นของ Held By 4',
       '% ที่ถือหุ้นของ Held By 5',
       'ชื่อกรรมการ 1', 'ชื่อกรรมการ 2', 'ชื่อกรรมการ 3', 'ชื่อกรรมการ 4',
       'ชื่อกรรมการ 5']]

In [33]:
# # columns V2
# df_enl1 = df_enl[[ 'เลขทะเบียนนิติบุคคล',
#        'ชื่อนิติบุคคล (ภาษาไทย)', 'ชื่อนิติบุคคล (ภาษาอังกฤษ)' , 'อำนาจกรรมการ' ,'% ที่ถือหุ้นของ Held By 1', '% ที่ถือหุ้นของ Held By 2',
#        '% ที่ถือหุ้นของ Held By 3', '% ที่ถือหุ้นของ Held By 4',
#        '% ที่ถือหุ้นของ Held By 5',
#        'ชื่อกรรมการ 1', 'ชื่อกรรมการ 2', 'ชื่อกรรมการ 3', 'ชื่อกรรมการ 4',
#        'ชื่อกรรมการ 5']]

In [34]:
share_col = []
director_col = []
col_list =  df_enl1.columns.tolist()
for i in range(len(col_list)):
  if 'ชื่อกรรมการ' in col_list[i]:
    director_col.append(col_list[i])
  elif 'ที่ถือหุ้นของ' in col_list[i]:
    share_col.append(col_list[i])

In [35]:
# id_list = df_enl1['เลขทะเบียนนิติบุคคล (13 หลัก)'].unique().tolist()
id_list = df_enl1['เลขทะเบียนนิติบุคคล (13 หลัก)'].unique().tolist()

In [36]:
df_enl2 = pd.DataFrame()
for i in range(len(id_list)):
  df_t = df_enl1[df_enl1['เลขทะเบียนนิติบุคคล (13 หลัก)'] == id_list[i]]
  df_t.reset_index(drop = True , inplace = True)
  director_name = []
  for m1 in range(len(director_col)):
    director_name.append(df_t.loc[0 , director_col[m1]])
  share_name = []
  for m2 in range(len(share_col)):
    share_name.append(df_t.loc[0,share_col[m2]])

  df_t1 = df_t.head(1)
  for m3 in range(len(director_col)):
    df_t1 = df_t1.drop(columns = [director_col[m3]])
  for m4 in range(len(share_col)):
    df_t1 = df_t1.drop(columns = [share_col[m4]])

  df_t1_1 = pd.DataFrame()
  df_t1_1['ชื่อผู้เกี่ยวข้อง'] = director_name
  df_t1_1['ความเกี่ยวข้อง'] = 'กรรมการ'


  df_t1_2 = pd.DataFrame()
  df_t1_2['ชื่อผู้เกี่ยวข้อง'] = share_name
  df_t1_2['ความเกี่ยวข้อง'] = 'ผู้ถือหุ้น'

  df_t2 = pd.concat([df_t1_1 , df_t1_2])
  df_t2.reset_index(drop = True , inplace = True)

  fill_col = ['เลขทะเบียนนิติบุคคล (13 หลัก)', 'ชื่อนิติบุคคลจดทะเบียนภาษาไทย', 'ชื่อนิติบุคคลจดทะเบียนภาษาอังกฤษ' , 'อำนาจกรรมการ' ]
  for n in range(len(fill_col)):
    df_t2[fill_col[n]] = df_t.loc[0,fill_col[n]]

  df_enl2 = pd.concat([df_enl2 , df_t2])

In [37]:
def clean_name_enl(a):
  if pd.isna(a) == False:
    if a == '-':
      a = np.nan
  return a

In [38]:
df_enl2['ชื่อผู้เกี่ยวข้อง'] = df_enl2['ชื่อผู้เกี่ยวข้อง'].apply(lambda x : clean_name_enl(x))

In [39]:
df_enl3 = df_enl2.dropna(subset = ['ชื่อผู้เกี่ยวข้อง'])

In [40]:
print(len(df_enl2) , len(df_enl3))

49210 28096


In [41]:
df_enl3.head()

,ชื่อผู้เกี่ยวข้อง,ความเกี่ยวข้อง,เลขทะเบียนนิติบุคคล (13 หลัก),ชื่อนิติบุคคลจดทะเบียนภาษาไทย,ชื่อนิติบุคคลจดทะเบียนภาษาอังกฤษ,อำนาจกรรมการ
0,นาง ปรัชญา อิทธิรส,กรรมการ,0505556011511,บริษัท 118 ทริปเปิ้ล พี จำกัด,"118 TRIPLE P CO.,LTD.",กรรมการหนึ่งคนลงลายมือชื่อและประทับตราสำคัญของ...
1,นาย ภาวิน อิทธิรส,กรรมการ,0505556011511,บริษัท 118 ทริปเปิ้ล พี จำกัด,"118 TRIPLE P CO.,LTD.",กรรมการหนึ่งคนลงลายมือชื่อและประทับตราสำคัญของ...
5,นางปรัชญา อิทธิรส (80.00%),ผู้ถือหุ้น,0505556011511,บริษัท 118 ทริปเปิ้ล พี จำกัด,"118 TRIPLE P CO.,LTD.",กรรมการหนึ่งคนลงลายมือชื่อและประทับตราสำคัญของ...
6,นายภาวิน อิทธิรส (18.75%),ผู้ถือหุ้น,0505556011511,บริษัท 118 ทริปเปิ้ล พี จำกัด,"118 TRIPLE P CO.,LTD.",กรรมการหนึ่งคนลงลายมือชื่อและประทับตราสำคัญของ...
7,นายสาธิต สว่างเรือง (1.25%),ผู้ถือหุ้น,0505556011511,บริษัท 118 ทริปเปิ้ล พี จำกัด,"118 TRIPLE P CO.,LTD.",กรรมการหนึ่งคนลงลายมือชื่อและประทับตราสำคัญของ...


In [42]:
def split_share_percent(a):
  b1 = np.nan
  b2 = np.nan
  if pd.isna(a) == False:
    b = a.rsplit(' ' , 1)
    b1 = b[0]
    b1 = b1.lower()
    b2 = b[1]
    clean_str1 = ['(' , ')' , '%' , ' ']
    for i in range(len(clean_str1)):
      b2 = b2.replace(clean_str1[i] , '')
    b2 = float(b2)
    clean_str2 = [' ','ด็อกเตอร์','ดร.','ว่าที่','พล.อ.','พล.ท.','พล.ต.','พ.อ.','พ.ท.','พ.ต.','ร.อ.','ร.ท.','ร.ต.','จ.ส.อ.','จ.ส.ท.','จ.ส.ต','ส.อ.',
                           'ส.ท.','ส.ต.',
                          'พลฯ','นนร.','พล.ร.อ.','พล.ร.ท.','พล.ร.ต.','น.อ.','ร.น.','น.ท.','น.ต.','ร.อ.','ร.ท.','ร.ต.','พ.จ.อ.','พ.จ.ท.','พ.จ.ต.',
                          'จ.อ.','จ.ท.','จ.ต.','พล.อ.อ.','พล.อ.ท.','พล.อ.ต.','พ.อ.อ.','พ.อ.ท.','พ.อ.ต.','นนอ.','นจอ.',
                          'พล.ต.อ.','พล.ต.ท.','พล.ต.ต.','พ.ต.อ.','พ.ต.ท.','พ.ต.ต.','ร.ต.อ.','ร.ต.ท.','ร.ต.ต.','ด.ต.','จ.ส.ต.','ส.ต.อ.','ส.ต.ท.','ส.ต.ต.'
                           ,'พลเอก','พลโท','พลตรี','พันเอก','พันโท','พันตรี','ร้อยเอก','ร้อยโท','ร้อยตรี','จ่าสิบเอก','จ่าสิบโท','จ่าสิบตรี','สิบเอก','สิบโท','สิบตรี','พลทหาร',
                        'พลเรือเอก','พลเรือโท','พลเรือตรี','นาวาเอก','นาวาโท','นาวาตรี','เรือเอก','เรือโท','เรือตรี','พันจ่าเอก','พันจ่าโท','พันจ่าตรี','จ่าเอก','จ่าโท','จ่าตรี',
                           'พลอากาศเอก' , 'พลอากาศโท','พลอากาศตรี','นาวาอากาศเอก','นาวาอากาศโท','นาวาอากาศตรี','เรืออากาศเอก','เรืออากาศโท' ,'เรืออากาศตรี',
                           'พันจ่าอากาศเอก','พันจ่าอากาศโท','พันจ่าอากาศตรี','จ่าอากาศเอก','จ่าอากาศโท','จ่าอากาศตรี',
                           'mr.','mrs.','ms.','limited','inc.','co., ltd.,','ltd','(ไทย)','(ประเทศไทย)',
                        '','(',')','-','.',',','บริษัท','บจก','จำกัด','มหาชน','ห้างหุ้นส่วน','จำกัด','publiccompanylim','company','–','/']
    for i in range(len(clean_str2)):
      b1 = b1.replace(clean_str2[i] , '')
  b1 = b1.replace('ฏ' , 'ฎ')
  return b1 , b2

In [43]:
df_s = df_enl3[df_enl3['ความเกี่ยวข้อง'] == 'ผู้ถือหุ้น']
df_d = df_enl3[df_enl3['ความเกี่ยวข้อง'] == 'กรรมการ']

In [44]:
df_s['Name_clean'] , df_s['เปอร์เซ็นผู้ถือหุ้น'] = zip(*df_s['ชื่อผู้เกี่ยวข้อง'].apply(lambda x : split_share_percent(x)))
df_s.head()

C:\Users\chaiyaphonp\AppData\Local\Temp\ipykernel_1968\3472303484.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_s['Name_clean'] , df_s['เปอร์เซ็นผู้ถือหุ้น'] = zip(*df_s['ชื่อผู้เกี่ยวข้อง'].apply(lambda x : split_share_percent(x)))
C:\Users\chaiyaphonp\AppData\Local\Temp\ipykernel_1968\3472303484.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_s['Name_clean'] , df_s['เปอร์เซ็นผู้ถือหุ้น'] = zip(*df_s['ชื่อผู้เกี่ยวข้อง'].apply(lambda x : split_share_percent(x)))


,ชื่อผู้เกี่ยวข้อง,ความเกี่ยวข้อง,เลขทะเบียนนิติบุคคล (13 หลัก),ชื่อนิติบุคคลจดทะเบียนภาษาไทย,ชื่อนิติบุคคลจดทะเบียนภาษาอังกฤษ,อำนาจกรรมการ,Name_clean,เปอร์เซ็นผู้ถือหุ้น
5,นางปรัชญา อิทธิรส (80.00%),ผู้ถือหุ้น,0505556011511,บริษัท 118 ทริปเปิ้ล พี จำกัด,"118 TRIPLE P CO.,LTD.",กรรมการหนึ่งคนลงลายมือชื่อและประทับตราสำคัญของ...,นางปรัชญาอิทธิรส,80.00
6,นายภาวิน อิทธิรส (18.75%),ผู้ถือหุ้น,0505556011511,บริษัท 118 ทริปเปิ้ล พี จำกัด,"118 TRIPLE P CO.,LTD.",กรรมการหนึ่งคนลงลายมือชื่อและประทับตราสำคัญของ...,นายภาวินอิทธิรส,18.75
7,นายสาธิต สว่างเรือง (1.25%),ผู้ถือหุ้น,0505556011511,บริษัท 118 ทริปเปิ้ล พี จำกัด,"118 TRIPLE P CO.,LTD.",กรรมการหนึ่งคนลงลายมือชื่อและประทับตราสำคัญของ...,นายสาธิตสว่างเรือง,1.25
5,นางปรัชญา อิทธิรส (81.60%),ผู้ถือหุ้น,0505561017123,บริษัท 118 เลเบอร์ แอนด์ ซัพพลาย แมนเนจเม้นท์ ...,"118 LABOR AND SUPPLY MANAGEMENT CO., LTD.",กรรมการหนึ่งคนลงลายมือชื่อ และประทับตราสำคัญขอ...,นางปรัชญาอิทธิรส,81.60
6,นายภาวิน อิทธิรส (18.40%),ผู้ถือหุ้น,0505561017123,บริษัท 118 เลเบอร์ แอนด์ ซัพพลาย แมนเนจเม้นท์ ...,"118 LABOR AND SUPPLY MANAGEMENT CO., LTD.",กรรมการหนึ่งคนลงลายมือชื่อ และประทับตราสำคัญขอ...,นายภาวินอิทธิรส,18.40


In [45]:
def clean_director(a):
    if pd.isna(a) == False:
        clean_str2 = [' ','ด็อกเตอร์','ดร.','ว่าที่','พล.อ.','พล.ท.','พล.ต.','พ.อ.','พ.ท.','พ.ต.','ร.อ.','ร.ท.','ร.ต.','จ.ส.อ.','จ.ส.ท.','จ.ส.ต','ส.อ.',
                           'ส.ท.','ส.ต.',
                          'พลฯ','นนร.','พล.ร.อ.','พล.ร.ท.','พล.ร.ต.','น.อ.','ร.น.','น.ท.','น.ต.','ร.อ.','ร.ท.','ร.ต.','พ.จ.อ.','พ.จ.ท.','พ.จ.ต.',
                          'จ.อ.','จ.ท.','จ.ต.','พล.อ.อ.','พล.อ.ท.','พล.อ.ต.','พ.อ.อ.','พ.อ.ท.','พ.อ.ต.','นนอ.','นจอ.',
                          'พล.ต.อ.','พล.ต.ท.','พล.ต.ต.','พ.ต.อ.','พ.ต.ท.','พ.ต.ต.','ร.ต.อ.','ร.ต.ท.','ร.ต.ต.','ด.ต.','จ.ส.ต.','ส.ต.อ.','ส.ต.ท.','ส.ต.ต.'
                           ,'พลเอก','พลโท','พลตรี','พันเอก','พันโท','พันตรี','ร้อยเอก','ร้อยโท','ร้อยตรี','จ่าสิบเอก','จ่าสิบโท','จ่าสิบตรี','สิบเอก','สิบโท','สิบตรี','พลทหาร',
                        'พลเรือเอก','พลเรือโท','พลเรือตรี','นาวาเอก','นาวาโท','นาวาตรี','เรือเอก','เรือโท','เรือตรี','พันจ่าเอก','พันจ่าโท','พันจ่าตรี','จ่าเอก','จ่าโท','จ่าตรี',
                           'พลอากาศเอก' , 'พลอากาศโท','พลอากาศตรี','นาวาอากาศเอก','นาวาอากาศโท','นาวาอากาศตรี','เรืออากาศเอก','เรืออากาศโท' ,'เรืออากาศตรี',
                           'พันจ่าอากาศเอก','พันจ่าอากาศโท','พันจ่าอากาศตรี','จ่าอากาศเอก','จ่าอากาศโท','จ่าอากาศตรี', 'เด็กหญิง', 'เด็กชาย',
                           'mr.','mrs.','ms.','limited','inc.','co., ltd.,','ltd','co., ltd','co.','(ไทย)','(ประเทศไทย)','(ประเทศไทย',
                        '–','(',')','-',',','บริษัท','บจก','จำกัด','มหาชน','ห้างหุ้นส่วน','จำกัด','publiccompanylim','company','–','/','นาย','นางสาว','น.ส.','.'  ,'นาง' , 'x0000' , '_' , 'และหรือย่อย', 'บมจ', 'หจก']
    else:
        return a
    for i in range(len(clean_str2)):
        a = a.replace(clean_str2[i] , '')
        a = a.replace('ฏ' , 'ฎ')
    return a
 

In [46]:
df_d['Name_clean'] = df_d['ชื่อผู้เกี่ยวข้อง'].apply(lambda x : clean_director(x) )

C:\Users\chaiyaphonp\AppData\Local\Temp\ipykernel_1968\3298249985.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_d['Name_clean'] = df_d['ชื่อผู้เกี่ยวข้อง'].apply(lambda x : clean_director(x) )


In [47]:
def clean_power(x):
  if pd.isna(x) == False:
    x = x.lower()
    x = x.strip()
    x = x.replace(' ','')
    str_level_list = ['ด็อกเตอร์','ดร.','ว่าที่','พล\.อ\.','พล\.ท\.','พล\.ต\.','พ\.อ\.','พ\.ท\.','พ\.ต\.','ร\.อ\.','ร\.ท\.','ร\.ต\.','จ\.ส\.อ.','จ\.ส\.ท.','จ\.ส\.ต','ส\.อ\.',
                           'ส\.ท\.','ส\.ต\.',
                          'พลฯ','นนร\.','พล\.ร\.อ\.','พล\.ร\.ท\.','พล\.ร\.ต.','น\.อ\.','ร\.น\.','น\.ท\.','น\.ต\.','ร\.อ\.','ร\.ท\.','ร\.ต\.','พ\.จ\.อ.','พ\.จ\.ท.','พ\.จ\.ต.',
                          'จ\.อ\.','จ\.ท\.','จ\.ต\.','พล\.อ\.อ.','พล\.อ\.ท\.','พล\.อ\.ต\.','พ\.อ\.อ\.','พ\.อ\.ท\.','พ\.อ\.ต\.','นนอ\.','นจอ\.',
                          'พล\.ต\.อ\.','พล\.ต\.ท\.','พล\.ต\.ต\.','พ\.ต\.อ\.','พ\.ต\.ท\.','พ\.ต\.ต\.','ร\.ต\.อ\.','ร\.ต\.ท\.','ร\.ต\.ต\.','ด\.ต\.','จ\.ส\.ต\.','ส\.ต\.อ\.','ส\.ต\.ท\.','ส\.ต\.ต\.'
                           ,'พลเอก','พลโท','พลตรี','พันเอก','พันโท','พันตรี','ร้อยเอก','ร้อยโท','ร้อยตรี','จ่าสิบเอก','จ่าสิบโท','จ่าสิบตรี','สิบเอก','สิบโท','สิบตรี','พลทหาร',
                        'พลเรือเอก','พลเรือโท','พลเรือตรี','นาวาเอก','นาวาโท','นาวาตรี','เรือเอก','เรือโท','เรือตรี','พันจ่าเอก','พันจ่าโท','พันจ่าตรี','จ่าเอก','จ่าโท','จ่าตรี',
                           'พลอากาศเอก' , 'พลอากาศโท','พลอากาศตรี','นาวาอากาศเอก','นาวาอากาศโท','นาวาอากาศตรี','เรืออากาศเอก','เรืออากาศโท' ,'เรืออากาศตรี',
                           'พันจ่าอากาศเอก','พันจ่าอากาศโท','พันจ่าอากาศตรี','จ่าอากาศเอก','จ่าอากาศโท','จ่าอากาศตรี','mr.','mrs.','ms.']
    det_str = ['mr.','mrs.','ms.','limited','inc.','co., ltd.,','ltd','(ไทย)','(ประเทศไทย)',
                        '','(',')','-','.',',','บริษัท','บจก','จำกัด','มหาชน','ห้างหุ้นส่วน','จำกัด','publiccompanylim','company','–','/']
    for i in range(len(str_level_list)):
      x = x.replace(str_level_list[i],'นาย')
    for i in range(len(det_str)):
      x = x.replace(det_str[i],'')
  x = x.replace('ฏ','ฎ')

  return x

In [48]:
df_d['อำนาจกรรมการ_clean'] = df_d['อำนาจกรรมการ'].apply(lambda x : clean_power(x))

C:\Users\chaiyaphonp\AppData\Local\Temp\ipykernel_1968\1870004402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_d['อำนาจกรรมการ_clean'] = df_d['อำนาจกรรมการ'].apply(lambda x : clean_power(x))


In [49]:
def check_director(a,b):
  if pd.isna(b) == True:
    c = np.nan
  elif a in b:
    c = 'มีอำนาจลงนาม'
  elif '(1)' in b:
    c = np.nan
  elif ( ('นาย' in b) or ('นาง' in b) ) and a not in b:
    c = 'ไม่มีอำนาจลงนาม'
  elif b == 'ไม่มี':
    c = np.nan
  else:
    c = 'มีอำนาจลงนาม'
  return c

In [50]:
df_d['อำนาจลงนาม'] = df_d.apply(lambda x : check_director(x['Name_clean'],x['อำนาจกรรมการ_clean']) , axis = 1)

C:\Users\chaiyaphonp\AppData\Local\Temp\ipykernel_1968\2444726065.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_d['อำนาจลงนาม'] = df_d.apply(lambda x : check_director(x['Name_clean'],x['อำนาจกรรมการ_clean']) , axis = 1)


In [51]:
df_cen['TH_NAME_CEN'] = df_cen['NAMETH'].fillna('') +' ' + df_cen['SURNAMETH'].fillna('')

In [52]:
df_cen1 = df_cen[['TH_NAME_CEN' , 'CUSTTYPE' , 'EXIM_CUSTID']].drop_duplicates()

In [53]:
df_cen1['Name_clean'] = df_cen1['TH_NAME_CEN'].apply(lambda x : clean_director(x))

In [54]:
df_cen1 = df_cen1.replace('' , np.nan)
df_cen1 = df_cen1.dropna(subset = ['Name_clean'])

In [55]:
def choose_from_cen(a):
  c = 0
  if pd.isna(a) == False:
    if ('กรรมการผู้มีอำนาจลงนาม' in a) or ('ผู้ถือหุ้นตั้งแต่ 20% ขึ้นไป' in a):
      c = 1
  return c

In [56]:
df_cen1['is_director'] = df_cen1['CUSTTYPE'].apply(lambda x : choose_from_cen(x))

In [57]:
df_cen2 = df_cen1[df_cen1['is_director'] == 1].drop(columns = ['is_director'])

In [58]:
def find_relation(a):
  c = np.nan
  if pd.isna(a) == False:
    if ('กรรมการผู้มีอำนาจลงนาม' in a) and ('ผู้ถือหุ้นตั้งแต่ 20% ขึ้นไป' in a):
      c = 'กรรมการ_cen/ผู้ถือหุ้น_cen'
    elif ('กรรมการผู้มีอำนาจลงนาม' in a):
      c = 'กรรมการ_cen'
    elif ('ผู้ถือหุ้นตั้งแต่ 20% ขึ้นไป' in a):
      c = 'ผู้ถือหุ้น_cen'
  return c

In [59]:
df_cen2['ความเกี่ยวข้อง'] = df_cen2['CUSTTYPE'].apply(lambda x : find_relation(x))

In [60]:
df_cen3 = pd.merge(df_cen2, df_pm , left_on = 'EXIM_CUSTID' , right_on = 'รหัสลูกค้า' , how = 'left')

In [61]:
df_cen3['CUCTAXCS'] = df_cen3['CUCTAXCS'].fillna(df_cen3['EXIM_CUSTID'])

In [62]:
df_cen4 = df_cen3[df_cen3['EXIM_CUSTID'].notna()]

In [63]:
df_cen4 = df_cen4.rename(columns = {'TH_NAME_CEN':'ชื่อผู้เกี่ยวข้อง' , 'CUCTAXCS':'เลขทะเบียนนิติบุคคล (13 หลัก)'})

In [64]:
df_cen5 = df_cen4[['ชื่อผู้เกี่ยวข้อง' , 'เลขทะเบียนนิติบุคคล (13 หลัก)' ,'ความเกี่ยวข้อง' , 'Name_clean']]

In [65]:
df_s1 = df_s[df_s['เปอร์เซ็นผู้ถือหุ้น'] >= 20]
# df_s1 = df_s.copy()

In [66]:
df_d1 = df_d[df_d['อำนาจลงนาม'] != 'ไม่มีอำนาจลงนาม']
#df_d1 = df_d.copy()

In [67]:
df_en1 = pd.concat([df_d1,df_s1,df_cen5])

In [68]:
print(len(df_en1) , len(df_d1) , len(df_s1) , len(df_cen5) , len(df_cen))

34871 8944 7683 18244 36252


In [69]:
df_en1 =df_en1.drop(columns = ['อำนาจกรรมการ_clean'])

In [70]:
df_en1 = df_en1.sort_values(['เลขทะเบียนนิติบุคคล (13 หลัก)' , 'ความเกี่ยวข้อง'])

In [71]:
df_en2 = pd.DataFrame()
cus_id = df_en1['เลขทะเบียนนิติบุคคล (13 หลัก)'].unique().tolist()
for i in range(len(cus_id)):
  df_t = df_en1[df_en1['เลขทะเบียนนิติบุคคล (13 หลัก)'] == cus_id[i]].copy()
  name_list = df_t['Name_clean'].unique().tolist()
  for j in range(len(name_list)):
    df_t1 = df_t[df_t['Name_clean'] == name_list[j]]
    df_t1.reset_index(drop = True , inplace = True)
    a1 = df_t1.loc[0,'ความเกี่ยวข้อง']
    a2 = df_t1.loc[0,'เปอร์เซ็นผู้ถือหุ้น']
    if len(df_t1) > 1:
      a2 = df_t1.loc[1,'เปอร์เซ็นผู้ถือหุ้น']
    for k in range(1 , len(df_t1)):
      a1 = a1 +'/' +df_t1.loc[k,'ความเกี่ยวข้อง']
    df_t1.loc[0,'ความเกี่ยวข้อง'] = a1
    df_t1.loc[0 , 'เปอร์เซ็นผู้ถือหุ้น'] = a2
    df_t2 = df_t1.head(1)
    df_en2 = pd.concat([df_en2 , df_t2])

In [72]:
df_en21 = df_en2.copy()

In [73]:
df_en2.columns

Index(['ชื่อผู้เกี่ยวข้อง', 'ความเกี่ยวข้อง', 'เลขทะเบียนนิติบุคคล (13 หลัก)',
       'ชื่อนิติบุคคลจดทะเบียนภาษาไทย', 'ชื่อนิติบุคคลจดทะเบียนภาษาอังกฤษ',
       'อำนาจกรรมการ', 'Name_clean', 'อำนาจลงนาม', 'เปอร์เซ็นผู้ถือหุ้น'],
      dtype='object')

In [74]:
id_list = df_en2['เลขทะเบียนนิติบุคคล (13 หลัก)'].unique().tolist()
df_en2 = pd.DataFrame()
fill_col = ['ชื่อนิติบุคคลจดทะเบียนภาษาไทย' , 'ชื่อนิติบุคคลจดทะเบียนภาษาอังกฤษ']
for i in range(len(id_list)):
  df_t = df_en21[df_en21['เลขทะเบียนนิติบุคคล (13 หลัก)'] == id_list[i]]
  df_t = df_t.sort_values(['ชื่อนิติบุคคลจดทะเบียนภาษาไทย'])
  df_t.reset_index(drop = True , inplace = True)
  if pd.isna(df_t.loc[0,'ชื่อนิติบุคคลจดทะเบียนภาษาไทย']) == False:
    for j in range(len(fill_col)):
      df_t[fill_col[j]] = df_t[fill_col[j]].fillna(df_t.loc[0,fill_col[j]])
  df_en2 = pd.concat([df_en2 , df_t])


In [75]:
print(len(df_en1) , len(df_en21) , len(df_en2))

34871 25642 25642


In [76]:
print(len(df_en1['Name_clean'].unique()) , len(df_en2['Name_clean'].unique()))

22035 22035


In [77]:
print(len(df_en1['เลขทะเบียนนิติบุคคล (13 หลัก)'].unique()) , len(df_en2['เลขทะเบียนนิติบุคคล (13 หลัก)'].unique()))

4984 4984


In [78]:
df_en2.reset_index(drop = True , inplace = True)

In [79]:
len(df_en2)

25642

## หาหัวกลุ่มที่จะใช้พิจารณา

In [80]:
def find_duplicates_name_diff_id(df_n3):
    name_2id = []
    name_list = list(set(df_n3['Name_clean']))
    # เอาบริษัทที่ชื่อไม่มีชื่อซ้ำออก
    for i in range(len(name_list)):
        df1 = df_n3[df_n3['Name_clean'] == name_list[i]]
        id_lst = list(set(df1['เลขทะเบียนนิติบุคคล (13 หลัก)']))
        if len(id_lst) != 1:
            name_2id.append(name_list[i])
    df_n4 = df_n3[df_n3['Name_clean'].isin(name_2id)]
    return df_n4

In [81]:
df_en3 = find_duplicates_name_diff_id(df_en2)

In [82]:
# df_hp = df_out4[['CUCTAXCS']].drop_duplicates()

In [83]:
df_h  = pd.merge(df_en3 , df_hp , left_on = 'เลขทะเบียนนิติบุคคล (13 หลัก)' , right_on = 'CUCTAXCS')

In [84]:
print(len(df_h) , len(df_en2) , len(df_en3))

2385 25642 6066


In [85]:
df_en3.reset_index(drop = True , inplace = True)

In [86]:
df_en3 = df_en3.replace('' , np.nan)

In [87]:
df_en3 = df_en3.dropna(subset = ['Name_clean'])

In [88]:
def find_group_relationship(df_head , df_name):

    id_head = list(set(df_head['เลขทะเบียนนิติบุคคล (13 หลัก)']))
    id_head.sort()

    df_a1 = pd.DataFrame(columns = df_name.columns)
    g = 1

    for i in range(len(id_head)):
        print(i)
        df1 = df_name[df_name['เลขทะเบียนนิติบุคคล (13 หลัก)'] == id_head[i]]

        name_lst = list(set(df1['Name_clean']))
        df1_1 = df_name[df_name['Name_clean'].isin(name_lst)]

        df2 = df1_1[df1_1['เลขทะเบียนนิติบุคคล (13 หลัก)'] != id_head[i]]

        id2_lst = list(set(df2['เลขทะเบียนนิติบุคคล (13 หลัก)']))
        c = 0
        for j in range(len(id2_lst)):
            df3 = df2[df2['เลขทะเบียนนิติบุคคล (13 หลัก)'] == id2_lst[j]]

            t = df3.copy()

            if c == 0:
                df1['กลุ่ม'] = g
                df1['สถานะ'] = 'HEAD'

                df_a1 = pd.concat([df_a1 , df1])

                t['กลุ่ม'] = g
                df_a1 = pd.concat([df_a1 , t])
                c = c+1
            else:
                t['กลุ่ม'] = g
                df_a1 = pd.concat([df_a1 , t])

        g = g+1

    return df_a1

In [89]:
df_a1 = find_group_relationship(df_h , df_en2)

0
1
2
3
4
5
6
7
8
9
10
11
12


C:\Users\chaiyaphonp\AppData\Local\Temp\ipykernel_1968\917492317.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['กลุ่ม'] = g
C:\Users\chaiyaphonp\AppData\Local\Temp\ipykernel_1968\917492317.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['สถานะ'] = 'HEAD'
C:\Users\chaiyaphonp\AppData\Local\Temp\ipykernel_1968\917492317.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

13
14
15
16
17
18
19
20
21
22
23
24
25


26
27
28
29
30
31
32
33


C:\Users\chaiyaphonp\AppData\Local\Temp\ipykernel_1968\917492317.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['กลุ่ม'] = g
C:\Users\chaiyaphonp\AppData\Local\Temp\ipykernel_1968\917492317.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['สถานะ'] = 'HEAD'
C:\Users\chaiyaphonp\AppData\Local\Temp\ipykernel_1968\917492317.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

...

C:\Users\chaiyaphonp\AppData\Local\Temp\ipykernel_1968\917492317.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['กลุ่ม'] = g
C:\Users\chaiyaphonp\AppData\Local\Temp\ipykernel_1968\917492317.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['สถานะ'] = 'HEAD'
C:\Users\chaiyaphonp\AppData\Local\Temp\ipykernel_1968\917492317.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

618
619
620
621
622
623
624
625
626
627
628
629
630
631
632
633
634
635
636
637
638


C:\Users\chaiyaphonp\AppData\Local\Temp\ipykernel_1968\917492317.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['กลุ่ม'] = g
C:\Users\chaiyaphonp\AppData\Local\Temp\ipykernel_1968\917492317.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['สถานะ'] = 'HEAD'
C:\Users\chaiyaphonp\AppData\Local\Temp\ipykernel_1968\917492317.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

639
640
641
642
643
644
645
646
647
648
649
650


C:\Users\chaiyaphonp\AppData\Local\Temp\ipykernel_1968\917492317.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['กลุ่ม'] = g
C:\Users\chaiyaphonp\AppData\Local\Temp\ipykernel_1968\917492317.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['สถานะ'] = 'HEAD'
C:\Users\chaiyaphonp\AppData\Local\Temp\ipykernel_1968\917492317.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

In [90]:
def delete_duplicates_group(df_a2_1):

    #df_a2_1['INDEX'] = np.nan

    g_list = list(set(df_a2_1['กลุ่ม']))
    g_list.sort()


    def find_index(df_a2_1):
        g = 0
        df_na1 = pd.DataFrame(columns = df_a2_1.columns)
        for i in range(len(g_list)):
            df_t1 = df_a2_1[df_a2_1['กลุ่ม'] == g_list[i] ]
            df_t1['INDEX'] = g
            df_na1 = pd.concat([df_na1 , df_t1])
            g = g+1

        return df_na1

    df_na1 = find_index(df_a2_1)

    df_na2 = df_na1[['เลขทะเบียนนิติบุคคล (13 หลัก)','Name_clean' , 'กลุ่ม']]
    df_na2 = df_na2.sort_values(['กลุ่ม','เลขทะเบียนนิติบุคคล (13 หลัก)','Name_clean'])
    p2_tuple = list(df_na2.itertuples(index=False, name=None))

    res = [tuple(v) for i, v in groupby(p2_tuple, lambda x: x[2])]
    new_list = list(tuple(x[0:2] for x in w) for w in res)
    dup_lst = [k for k,v in Counter(new_list).items() if v>1]
    dup_in = []

    for j in range(len(dup_lst)):
        index = [i for i, e in enumerate(new_list) if e == dup_lst[j]]
        dup_in.append(index)

    # เอาแค่ตัวแรกเก็บไว้
    dup_in2 = list(x[0] for x in dup_in)

    def flatten(t):
        return [item for sublist in t for item in sublist]

    dup_in_f =  flatten(dup_in)

    df_not_dup = df_na1[~df_na1['INDEX'].isin(dup_in_f )]
    df_dup = df_na1[df_na1['INDEX'].isin(dup_in_f )]
    df_not_dup2 = df_dup[df_dup['INDEX'].isin(dup_in2)]

    df_re1 = pd.concat([df_not_dup,df_not_dup2])

    return df_re1

In [91]:
df_a2 = delete_duplicates_group(df_a1)

C:\Users\chaiyaphonp\AppData\Local\Temp\ipykernel_1968\3170041409.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_t1['INDEX'] = g
C:\Users\chaiyaphonp\AppData\Local\Temp\ipykernel_1968\3170041409.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_t1['INDEX'] = g
C:\Users\chaiyaphonp\AppData\Local\Temp\ipykernel_1968\3170041409.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

C:\Users\chaiyaphonp\AppData\Local\Temp\ipykernel_1968\3170041409.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_t1['INDEX'] = g
C:\Users\chaiyaphonp\AppData\Local\Temp\ipykernel_1968\3170041409.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_t1['INDEX'] = g
C:\Users\chaiyaphonp\AppData\Local\Temp\ipykernel_1968\3170041409.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

In [92]:
print(len(df_a1) , len(df_a2))

7664 7218


In [93]:
df_a2.head()

,ชื่อผู้เกี่ยวข้อง,ความเกี่ยวข้อง,เลขทะเบียนนิติบุคคล (13 หลัก),ชื่อนิติบุคคลจดทะเบียนภาษาไทย,ชื่อนิติบุคคลจดทะเบียนภาษาอังกฤษ,อำนาจกรรมการ,Name_clean,อำนาจลงนาม,เปอร์เซ็นผู้ถือหุ้น,กลุ่ม,สถานะ,INDEX
62,วัฒนา ชมชาญ,กรรมการ_cen,0009988,NaN,NaN,NaN,วัฒนาชมชาญ,NaN,NaN,1.0,HEAD,0.0
63,เพชร์ พฤกษาทร,กรรมการ_cen,0009988,NaN,NaN,NaN,เพชร์พฤกษาทร,NaN,NaN,1.0,HEAD,0.0
64,ธีระพล พฤกษาทร,กรรมการ_cen/ผู้ถือหุ้น_cen,0009988,NaN,NaN,NaN,ธีระพลพฤกษาทร,NaN,NaN,1.0,HEAD,0.0
65,ชาการ่า อินเตอร์เนชั่นแนล,ผู้ถือหุ้น_cen,0009988,NaN,NaN,NaN,ชาการ่าอินเตอร์เนชั่นแนล,NaN,NaN,1.0,HEAD,0.0
66,ซูเทค เอ็นจิเนียริ่ง,ผู้ถือหุ้น_cen,0009988,NaN,NaN,NaN,ซูเทคเอ็นจิเนียริ่ง,NaN,NaN,1.0,HEAD,0.0


In [94]:
df_pm.head()

,รหัสลูกค้า,CUCTAXCS
0,G020109,0105538015709
1,G021226,0105514002631
2,G021476,0107537000262
3,G021488,0103521001856
4,G021616,0105544012597


In [95]:
df_a3 = pd.merge(df_a2 , df_pm , left_on = 'เลขทะเบียนนิติบุคคล (13 หลัก)' , right_on = 'CUCTAXCS' , how = 'left')

In [96]:
df_a3 = df_a3.rename(columns = {'รหัสลูกค้า':'รหัสลูกค้า_en'})

In [97]:
df_a3['รหัสลูกค้า_en'] = df_a3['รหัสลูกค้า_en'].fillna(df_a3['เลขทะเบียนนิติบุคคล (13 หลัก)'])

In [98]:
# df_a3[df_a3['รหัสลูกค้า_en'].isna()]

In [99]:
df_a3 = df_a3.drop(columns = ['CUCTAXCS'])

## Merge File กลุ่ม

In [100]:
df_g1 = df_g[['exim_groupcode' , 'name']].drop_duplicates()

In [101]:
df_a4 = pd.merge(df_a3 , df_g1 , left_on = 'รหัสลูกค้า_en' , right_on = 'name' , how = 'left' )

In [102]:
df_a4.head()

,ชื่อผู้เกี่ยวข้อง,ความเกี่ยวข้อง,เลขทะเบียนนิติบุคคล (13 หลัก),ชื่อนิติบุคคลจดทะเบียนภาษาไทย,ชื่อนิติบุคคลจดทะเบียนภาษาอังกฤษ,อำนาจกรรมการ,Name_clean,อำนาจลงนาม,เปอร์เซ็นผู้ถือหุ้น,กลุ่ม,สถานะ,INDEX,รหัสลูกค้า_en,exim_groupcode,name
0,วัฒนา ชมชาญ,กรรมการ_cen,0009988,NaN,NaN,NaN,วัฒนาชมชาญ,NaN,NaN,1.0,HEAD,0.0,0009988,R0033,0009988
1,เพชร์ พฤกษาทร,กรรมการ_cen,0009988,NaN,NaN,NaN,เพชร์พฤกษาทร,NaN,NaN,1.0,HEAD,0.0,0009988,R0033,0009988
2,ธีระพล พฤกษาทร,กรรมการ_cen/ผู้ถือหุ้น_cen,0009988,NaN,NaN,NaN,ธีระพลพฤกษาทร,NaN,NaN,1.0,HEAD,0.0,0009988,R0033,0009988
3,ชาการ่า อินเตอร์เนชั่นแนล,ผู้ถือหุ้น_cen,0009988,NaN,NaN,NaN,ชาการ่าอินเตอร์เนชั่นแนล,NaN,NaN,1.0,HEAD,0.0,0009988,R0033,0009988
4,ซูเทค เอ็นจิเนียริ่ง,ผู้ถือหุ้น_cen,0009988,NaN,NaN,NaN,ซูเทคเอ็นจิเนียริ่ง,NaN,NaN,1.0,HEAD,0.0,0009988,R0033,0009988


In [103]:
# df_a4.to_excel(output_path +file_name , index = False)

## เอาข้อมูลที่ถูกต้องแล้วออก

In [104]:
df_a5 = pd.DataFrame()
g_list = df_a4['กลุ่ม'].unique().tolist()
for i in range(len(g_list)):
  df_t = df_a4[df_a4['กลุ่ม'] == g_list[i]]
  name_list = df_t['Name_clean'].unique().tolist()
  for j in range(len(name_list)):
    df_t1 = df_t[df_t['Name_clean'] == name_list[j]]
    t1 = df_t1[df_t1['exim_groupcode'].isna()]
    t2 = df_t1['exim_groupcode'].unique().tolist()
    t3 = df_t1['เลขทะเบียนนิติบุคคล (13 หลัก)'].unique().tolist()
    if ((len(t1) > 0) or (len(t2) > 1)) and (len(t3) > 1):
      id_mlist = df_t1['เลขทะเบียนนิติบุคคล (13 หลัก)'].unique().tolist()
      s0 = set(list(df_t1[df_t1['เลขทะเบียนนิติบุคคล (13 หลัก)'] == id_mlist[0]]['exim_groupcode']))
      c = 0
      for k in range(1,len(id_mlist)):
        df_t2 = df_t1[df_t1['เลขทะเบียนนิติบุคคล (13 หลัก)'] == id_mlist[k]]
        s1 = set(list(df_t2['exim_groupcode']))
        s2 = s1.intersection(s1)
        if len(s2) == 0:
          c+=1
      if (c != 0) or (len(t1) > 0):
        df_a5 = pd.concat([df_a5 , df_t1])

In [105]:
print(len(df_a4) , len(df_a5))

7750 189


# EXPORT FILE

In [106]:
if len(df_a5) != 0:
  df_a6 = pd.merge(df_a5 , df_out2 , left_on = 'รหัสลูกค้า_en' , right_on = 'รหัสลูกค้า' , how = 'left')
  df_a6 = df_a6.sort_values(['กลุ่ม','รหัสลูกค้า_en'])
  df_a6.to_excel(output_path + file_name , index = False)
else:
  df_a5.to_excel(output_path + file_name , index = False)